In [ ]:
LANGCHAIN_TRACING_V2="true"
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="api_key"
LANGCHAIN_PROJECT="Zero2LLMOps"

In [2]:
import os

os.environ['LANGCHAIN_TRACING_V2'] = LANGCHAIN_TRACING_V2
os.environ['LANGCHAIN_ENDPOINT'] = LANGCHAIN_ENDPOINT
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY
os.environ['LANGCHAIN_PROJECT'] = LANGCHAIN_PROJECT

In [7]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(
    file_path="./transformersPaper.pdf"
)

docs = loader.load()

In [13]:
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter
)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

In [14]:
from langchain_ollama.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)

from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(
    documents=splits,
    embedding=embeddings
)

In [15]:
retriver = vector_store.as_retriever()

In [16]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain =(
    RunnableParallel({"context": retriver | format_docs, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
rag_chain.invoke("What is the Author of Transformer paper ??")

'The author of the Transformer paper is Ashish Vaswani, along with co-authors Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.'

In [18]:
rag_chain.invoke("what Transformer paper want to say ??")

'The Transformer paper wants to say that it proposes a new architecture for sequence-to-sequence tasks, replacing RNNs with self-attention and achieving state-of-the-art results on several benchmarks. It also shows that the model generalizes well to other tasks, such as English constituency parsing.'

In [19]:
rag_chain.invoke("what is the ideology behind the  Transformer paper ??")

'The ideology behind the Transformer paper is to propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions. This approach aims to simplify sequence transduction models by eliminating complex recurrent or convolutional neural networks.'

In [20]:
rag_chain.invoke("whats your thoughts on transformer paper btw ??")

"I don't know."

In [21]:
rag_chain.invoke("what is the disadvantages of tansformer paper  ??")

"I don't know."

# **Query Transformation**

- ## **Multi Query**

In [22]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [24]:
generate_queries.invoke("What is RAG??")

['Here are five different versions of the original question:',
 '',
 'What is the definition or explanation for "RAG"?',
 '',
 'Can you provide information about what "RAG" stands for?',
 '',
 'What are the key characteristics, features, or properties of something related to "RAG"?',
 '',
 'Is there a specific context or domain where "RAG" is commonly used or referenced?',
 '',
 'What are some possible synonyms, alternatives, or related concepts to "RAG"?']

In [25]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "whats your thoughts on transformer paper btw ??"
retrieval_chain = generate_queries | retriver.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

14

In [26]:
docs

[Document(metadata={'source': './transformersPaper.pdf', 'page': 11}, page_content='[37] Vinyals & Kaiser, Koo, Petrov, Sutskever, and Hinton. Grammar as a foreign language. In\nAdvances in Neural Information Processing Systems, 2015.\n[38] Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V Le, Mohammad Norouzi, Wolfgang\nMacherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google’s neural machine\ntranslation system: Bridging the gap between human and machine translation. arXiv preprint\narXiv:1609.08144, 2016.\n[39] Jie Zhou, Ying Cao, Xuguang Wang, Peng Li, and Wei Xu. Deep recurrent models with\nfast-forward connections for neural machine translation. CoRR, abs/1606.04199, 2016.\n[40] Muhua Zhu, Yue Zhang, Wenliang Chen, Min Zhang, and Jingbo Zhu. Fast and accurate\nshift-reduce constituent parsing. In Proceedings of the 51st Annual Meeting of the ACL (Volume\n1: Long Papers), pages 434–443. ACL, August 2013.\n12'),
 Document(metadata={'source': './transformersPaper.pdf',

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})


"I don't have personal thoughts or opinions like humans do, but I can provide an analysis of the Transformer paper based on its content.\n\nThe Transformer paper, published in 2017 by Vaswani et al., revolutionized the field of natural language processing (NLP) and machine learning. It introduced a novel architecture that replaced traditional recurrent neural networks (RNNs) with self-attention mechanisms, achieving state-of-the-art results in various NLP tasks.\n\nHere are some key takeaways from the paper:\n\n1. **Self-Attention Mechanism**: The Transformer's core innovation is the self-attention mechanism, which allows the model to weigh and combine different input elements (e.g., words or tokens) based on their relevance to each other.\n2. **Parallelization**: Unlike RNNs, which process sequences sequentially, Transformers can process entire sequences in parallel, making them much faster and more efficient.\n3. **Multi-Head Attention**: The Transformer uses a multi-head attention m